# Import External Modules

In [ ]:
# Gym & Gym_Anytrading
import gym
import gym.vector
import gym_anytrading

# Pytorch Modules
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.cuda.amp import GradScaler, autocast

# Data Structures
import numpy as np
import pandas as pd
from collections import deque

# Data Science
import statsmodels.api as sm
from ta.momentum import RSIIndicator
from ta.trend import MACD
from sklearn.linear_model import HuberRegressor, LinearRegression
from sklearn.preprocessing import StandardScaler
from numba import jit, prange

# Misc
from tqdm import tqdm
import os

# Data processing functions

### Calculate Rolling Standard Deviation

For a window size of $w$ and time index $i$, the rolling standard deviation at index $i + w - 1$ is calculated as:

$$\sigma_{i+w-1} = \sqrt{\frac{1}{w} \sum_{j=i}^{i+w-1} (x_j - \bar{x})^2}$$

Where:
- $x_j$ represents the array values at time $j$
- $\bar{x}$ is the mean of the array values in the current window: $\bar{x} = \frac{1}{w} \sum_{j=i}^{i+w-1} x_j$
- $w$ is the window size

The function calculates this standard deviation for each possible window of size $w$ in the input array, storing the result at the end position of each window in the output array.

Note that the output array values at indices $0$ through $w-2$ remain unmodified by this function, as there aren't enough preceding values to form a complete window.

In [ ]:
@jit(nopython=True)
def calculate_rolling_std(arr, output, window):
  n = len(arr)
  for i in range(n - window + 1):
      output[i + window - 1] = np.std(arr[i:i + window])
  return output

### Calculate Least Squares
The function implements the analytical solution to the least squares problem for a linear regression with two parameters:

$$\hat{\beta} = (X^T X)^{-1} X^T y$$

Where:
- $X$ is the design matrix of size $n \times 2$ (first column of ones, second column of predictor values)
- $y$ is the response vector of size $n$
- $\hat{\beta}$ is the vector of estimated coefficients $[\beta_0, \beta_1]$

For the 2×2 matrix inversion, the function calculates:

$$\text{det} = (X^T X)_{00} \cdot (X^T X)_{11} - (X^T X)_{01} \cdot (X^T X)_{10}$$

If $|\text{det}| < 10^{-10}$, the function returns $[\text{NaN}, \text{NaN}]$ as the matrix is considered non-invertible.

Otherwise, the inverse of $X^T X$ is calculated as:

$$(X^T X)^{-1} = \frac{1}{\text{det}} \begin{bmatrix} (X^T X)_{11} & -(X^T X)_{01} \\ -(X^T X)_{10} & (X^T X)_{00} \end{bmatrix}$$

Finally, the function returns:

$$\hat{\beta} = (X^T X)^{-1} X^T y$$

This implementation is optimized for a 2×2 matrix inversion within the Numba JIT environment.

In [ ]:
@jit(nopython=True)
def calc_lstsq(X, y):
  # Solve X'X b = X'y
  XTX = X.T @ X
  XTy = X.T @ y

  # Check if XTX is invertible
  det = XTX[0, 0] * XTX[1, 1] - XTX[0, 1] * XTX[1, 0]
  if abs(det) < 1e-10:
      return np.array([np.nan, np.nan])

  # Inverse of 2x2 matrix
  inv_XTX = np.empty((2, 2))
  inv_XTX[0, 0] = XTX[1, 1] / det
  inv_XTX[1, 1] = XTX[0, 0] / det
  inv_XTX[0, 1] = -XTX[0, 1] / det
  inv_XTX[1, 0] = -XTX[1, 0] / det

  # Calculate beta
  beta = inv_XTX @ XTy
  return beta

### Calculate Rolling Cointegration

For a window size of $w$ and time index $i$, the cointegration score at index $i + w - 1$ is calculated as:

$$\text{score}_{i+w-1} = -\frac{\sigma(\epsilon)}{\mu(|\epsilon|)}$$

Where:
- $\epsilon$ are the residuals from the linear regression: $\epsilon = y - (β_0 + β_1 \cdot x)$
- $\sigma(\epsilon)$ is the standard deviation of the residuals
- $\mu(|\epsilon|)$ is the mean of the absolute values of the residuals
- $β_0$ and $β_1$ are regression coefficients found using least squares

The regression coefficients are calculated as:

$$[β_0, β_1] = \text{argmin}_β \sum_{j=i}^{i+w-1} (y_j - β_0 - β_1 \cdot x_j)^2$$

Where:
- $y_j$ represents asset1 prices at time $j$
- $x_j$ represents asset2 prices at time $j$
- $w$ is the window size (60 by default in the code)

The score is a simplified approximation of the Augmented Dickey-Fuller (ADF) test statistic, which is used to test for stationarity in the residuals as a measure of cointegration between the two asset price series.

The calculation is only performed when:
- The standard deviation of both series within the window exceeds $\epsilon$ (1e-10)
- The standard deviation of the residuals exceeds $\epsilon$

In [ ]:
@jit(nopython=True)
def rolling_cointegration_score(asset1, asset2, window=60, eps=1e-10):
  n = len(asset1)
  if n < window:
    raise ValueError("Window size too large for available data")

  scores = np.full(n, np.nan)

  for i in range(n - window + 1):
    y = asset1[i:i + window]
    x_vals = asset2[i:i + window]

    # Check if there's enough variation in both series
    y_std = np.std(y)
    x_std = np.std(x_vals)

    if y_std < eps or x_std < eps:
      continue

    # Create design matrix
    X = np.column_stack((np.ones(window), x_vals))

    # Calculate coefficients
    beta = calc_lstsq(X, y)

    # If calculation failed, skip
    if np.isnan(beta[0]):
      continue

    # Calculate residuals
    residuals = y - (beta[0] + beta[1] * x_vals)

    # Check if residuals are essentially flat
    if np.std(residuals) < eps:
      continue

    # Simplified ADF test statistic (using residual variance)
    # This is a proxy for the ADF test since full ADF test is complex for numba
    adf_stat = -np.std(residuals) / np.mean(np.abs(residuals))
    scores[i + window - 1] = adf_stat

  return scores

### Calculate Rolling Hedge Ratio

For a window size of $n$ and time index $i$, the hedge ratio $\beta_i$ is calculated as:

$$\beta_i = \frac{\sum_{j=i-n+1}^{i}(x_j - \bar{x})(y_j - \bar{y})}{\sum_{j=i-n+1}^{i}(x_j - \bar{x})^2}$$

Where:
- $x_j$ represents asset2 prices at time $j$
- $y_j$ represents asset1 prices at time $j$
- $\bar{x}$ is the mean of asset2 prices in the current window
- $\bar{y}$ is the mean of asset1 prices in the current window
- $n$ is the window size (30 by default in the code)

This formula calculates the slope (beta) of the linear regression between the two asset price series within each rolling window, which represents the optimal hedge ratio.

In [ ]:
@jit(nopython=True)
def calculate_rolling_hedge_ratio_numba(asset1_prices, asset2_prices, window_size=30):
  n = len(asset1_prices)
  hedge_ratios = np.zeros(n)

  # Fill with NaN for the first window_size-1 elements
  hedge_ratios[:window_size-1] = np.nan

  for i in range(window_size-1, n):
    # Get the window of data
    y = asset1_prices[i-window_size+1:i+1]
    x = asset2_prices[i-window_size+1:i+1]

    # Calculate means
    x_mean = np.mean(x)
    y_mean = np.mean(y)

    # Calculate the numerator and denominator for the hedge ratio
    numerator = 0.0
    denominator = 0.0

    for j in range(window_size):
      x_diff = x[j] - x_mean
      y_diff = y[j] - y_mean
      numerator += x_diff * y_diff
      denominator += x_diff * x_diff

    # Calculate hedge ratio (beta)
    if denominator != 0:
      hedge_ratios[i] = numerator / denominator
    else:
      hedge_ratios[i] = np.nan

  return hedge_ratios

### Calculate Log Returns

For a price series $P = [P_0, P_1, ..., P_n]$, the logarithmic return at time $t$ is calculated as:

$$r_t = \ln(P_t) - \ln(P_{t-1})$$

Where:
- $r_t$ represents the logarithmic return at time $t$
- $P_t$ represents the price at time $t$
- $P_{t-1}$ represents the price at time $t-1$
- $\ln()$ is the natural logarithm function

This formula calculates the continuously compounded return between two consecutive price observations. Log returns have the advantage of being additive across time and approximately equal to simple returns for small price changes.

In the implementation, this is achieved by applying the natural logarithm to the entire price series and then calculating the first difference of the resulting series.

In [ ]:
def calculate_log_returns(prices):
  return np.log(prices).diff()

# Process Training Data

### Import & format datetime

In [ ]:
eudf = pd.read_csv("./Data/EURUSD.csv", index_col='Gmt time')
gbdf = pd.read_csv("./Data/GBPUSD.csv", index_col='Gmt time')
eudf["Gmt time"] = pd.to_datetime(eudf["Gmt time"], format="%Y-%m-%d %H:%M:%S")
gbdf["Gmt time"] = pd.to_datetime(gbdf["Gmt time"], format="%Y-%m-%d %H:%M:%S")

### Create price columns

In [ ]:
ptd = pd.DataFrame(index=eudf.index)
ptd["Asset1_Price"] = eudf["Close"]
ptd["Asset2_Price"] = gbdf["Close"]

ptd["Open"] = eudf['Open'] / gbdf['Open']
ptd["High"] = eudf['High'] / gbdf['High']
ptd["Low"] = eudf['Low'] / gbdf['Low']
ptd["Close"] = eudf['Close'] / gbdf['Close']
ptd['Ratio_Price'] = eudf['Close'] / gbdf['Close']

### Calculate hedge ratio column

In [ ]:
ptd["Hedge_Ratio"] = calculate_rolling_hedge_ratio_numba(ptd["Asset1_Price"], ptd["Asset2_Price"])

### Calculate spread z-score

In [ ]:
spread = ptd["Asset1_Price"] - ptd["Asset2_Price"]
ptd["Spread_ZScore"] = (spread - spread.rolling(30).mean()) / spread.rolling(30).std()

### Calculate correlation & cointegration

In [ ]:
ptd["Rolling_Correlation"] = ptd["Asset1_Price"].rolling(30).corr(ptd["Asset2_Price"])
coint_scores = rolling_cointegration_score(ptd["Asset1_Price"].values, ptd["Asset2_Price"].values)
ptd["Rolling_Cointegration_Score"] = coint_scores

### Calculate RSI

In [ ]:
ptd["RSI1"] = RSIIndicator(ptd["Asset1_Price"], window=14).rsi()
ptd["RSI2"] = RSIIndicator(ptd["Asset2_Price"], window=14).rsi()
ptd["RSI3"] = RSIIndicator(ptd["Ratio_Price"], window=14).rsi()

### Calculate MACD

In [ ]:
ptd["MACD1"] = MACD(ptd["Asset1_Price"], window_slow=26, window_fast=12, window_sign=9).macd()
ptd["MACD2"] = MACD(ptd["Asset2_Price"], window_slow=26, window_fast=12, window_sign=9).macd()
ptd["MACD3"] = MACD(ptd["Ratio_Price"], window_slow=26, window_fast=12, window_sign=9).macd()

### Last touches

In [ ]:
ptd.dropna(inplace=True)
ptd.to_csv("./data/processed/train_data.csv")

### Import data using Pandas
```python
eData = pd.read_csv("data/EURUSDp.csv")
...
```

### Format datetime
```python
eData["Gmt time"] = pd.to_datetime(eData["Gmt time"], format="%Y-%m-%d %H:%M:%S")
...
```

### Rename Columns
```python
eData['open'], eData['high'], eData['low'], eData['close'] = eData['Open'], eData['High'], eData['Low'], eData['Close']
...
```

### Set Index
```python
eData.set_index("Gmt Time", inplace=True)
...
```

In [ ]:
eData = pd.read_csv("data/EURUSDp.csv")
pData = pd.read_csv("data/GBPUSDp.csv")
xData = pd.read_csv("data/processed/EXTRAp.csv")

eData["Gmt time"] = pd.to_datetime(eData["Gmt time"], format="%Y-%m-%d %H:%M:%S")
pData["Gmt time"] = pd.to_datetime(pData["Gmt time"], format="%Y-%m-%d %H:%M:%S")
xData["Gmt time"] = pd.to_datetime(xData["Gmt time"], format="%Y-%m-%d %H:%M:%S")

eData['open'], eData['high'], eData['low'], eData['close'] = eData['Open'], eData['High'], eData['Low'], eData['Close']
pData['open'], pData['high'], pData['low'], pData['close'] = pData['Open'], pData['High'], pData['Low'], pData['Close']

eData.set_index("Gmt Time", inplace=True)
pData.set_index("Gmt Time", inplace=True)
xData.set_index("Gmt Time", inplace=True)

# Environment Functions

### Reward Function

In [ ]:
def reward_function():
  pass

### Dynamic Enviroment Data

In [ ]:
def dynamic_feature_positioning(history):
  return history

def dynamic_feature_unrealized(history):
  return history

def dynamic_feature_realized(history):
  return history

### make_function(n: int)
takes in an ID number `n` then return an environment with the correct information

In [ ]:
def make_function(n: int):
  match n:
    case 1:
      return gym.make(
        id="forex-v0",
        name="EUR/USD",
        df=eData,
        dynamic_feature_functions = [],
        reward_function=reward_function,
        verbose=True
       )
    case 2:
      return gym.make(
        id="forex-v0",
        name="EUR/USD",
        df=pData,
        dynamic_feature_functions = [],
        reward_function=reward_function,
        verbose=True
       )
    case 3:
      return gym.make(
        id="forex-v0",
        name="EXTRA",
        df=xData,
        dynamic_feature_functions = [],
        reward_function=reward_function,
        verbose=True
       )

# LSTM Deep Q Learning Model
Pytorch LSTM Model for Deep Q Learning

## Constructor
```python
LSTM_Q_Net(input_size: int, hidden_size: int, output_size: int)
```
### Inputs
- `input_size`: Number of input features
- `hidden_size`: Number of input features
- `output_size`: Number of input features

## Overview
```python
forward(x: torch.tensor)

save(file_name: str)
```


In [4]:
class LSTM_Q_Net(nn.Module):
  def __init__(self, input_size, hidden_size, output_size):
    super(LSTM_Q_Net, self).__init__()
    self.hidden_size = hidden_size
    self.lstm = nn.LSTM(
      input_size=input_size,
      hidden_size=hidden_size,
      num_layers=2,
      batch_first=True,
    )
    self.fc2 = nn.Linear(hidden_size, output_size)
    self._init_weights()

  def forward(self, x):
    out, hidden = self.lstm(x, hidden)
    out = self.fc(out[:, -1, :])
    return out, hidden

  def _init_weights(self):
    for name, param in self.named_parameters():
      if 'weight' in name:
        if 'lstm' in name:
          nn.init.xavier_uniform_(param)
        else:
          nn.init.xavier_uniform_(param)
      elif 'bias' in name:
        nn.init.zeros_(param)

  def save(self, file_name):
    model_folder_path = "./models"
    os.makedirs(model_folder_path, exist_ok=True)
    file_path = os.path.join(model_folder_path, file_name)
    torch.save(self.state_dict(), file_path)

NameError: name 'nn' is not defined

## Define Training Agent

In [ ]:
class TrainingAgent:
  def __init__(self, envs, hidden_size=128, input_size=16, output_size=4, learning_rate=1e-3,
               gamma=0.99, epsilon=1.0, epsilon_min=0.01, epsilon_decay=0.995,
               seq_len=32, batch_size=64, memory_size=10000):

    self.envs = envs

    self.input_size = input_size
    self.output_size = output_size
    self.hidden_size = hidden_size

    self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    self.model = LSTM_Q_Net(self.input_size, hidden_size, self.output_size).to(self.device)
    self.target_model = LSTM_Q_Net(self.input_size, hidden_size, self.output_size).to(self.device)
    self.target_model.load_state_dict(self.model.state_dict())

    self.criterion = nn.MSELoss()
    self.optimizer = optim.Adam(self.model.parameters(), lr=learning_rate)
    self.scaler = GradScaler(device=self.device.type)

    self.gamma = gamma
    self.epsilon = epsilon
    self.epsilon_min = epsilon_min
    self.epsilon_decay = epsilon_decay

    self.memory = deque(maxlen=memory_size)
    self.seq_len = seq_len
    self.batch_size = batch_size

# Define TBPTT()

In [ ]:
def TBPTT(memory):
  if not memory:
    return None, None, None, None, None, None

  states, actions, rewards, next_states, dones, hidden_states = zip(*memory)

  device = hidden_states[0][0].device if hidden_states else torch.device("cpu")

  states = torch.tensor(np.array(states), dtype=torch.float32).to(device)
  actions = torch.tensor(actions, dtype=torch.int64).to(device).unsqueeze(1)
  rewards = torch.tensor(rewards, dtype=torch.float32).to(device).unsqueeze(1)
  next_states = torch.tensor(np.array(next_states), dtype=torch.float32).to(device)
  dones = torch.tensor(dones, dtype=torch.float32).to(device).unsqueeze(1)

  h_states = (torch.cat([h[0] for h in hidden_states], dim=1),
              torch.cat([h[1] for h in hidden_states], dim=1))

  return states, next_states, h_states, actions, rewards, dones

# Utility Functions

In [ ]:
def expand(state):
  if isinstance(state, np.ndarray):
    return np.expand_dims(state, axis=0), np.expand_dims(state, axis=0), state
  else:
    state_array = np.array(state)
    return np.expand_dims(state_array, axis=0), np.expand_dims(state_array, axis=0), state_array

# Define train_on_batch

In [ ]:
def train_on_batch(agent: TrainingAgent, memory):
  if len(memory) == 0:
    return

  states, next_states, hidden_states, actions, rewards, dones = TBPTT(memory)

  with torch.no_grad(), autocast(device_type=agent.device.type, dtype=torch.float16):
    target_q_values, _ = agent.target_model(next_states, hidden_states)
    max_next_q_values = torch.max(target_q_values, dim=1, keepdim=True)[0]
    target_q = rewards + (1 - dones) * agent.gamma * max_next_q_values

  agent.optimizer.zero_grad()
  with autocast(device_type=agent.device.type, dtype=torch.float16):
    current_q_values, _ = agent.model(states, hidden_states)
    current_q_values = current_q_values.gather(1, actions)
    loss = agent.criterion(current_q_values, target_q)

  agent.scaler.scale(loss).backward()
  agent.scaler.step(agent.optimizer)
  agent.scaler.update()

# Define run_episode()

In [ ]:
def run_episode(agent: TrainingAgent, render=False):
  state = agent.envs.reset()
  state0, state1, state2 = expand(state)

  hidden_state = (torch.zeros(agent.num_layers, 1, agent.hidden_size, dtype=torch.float16).to(agent.device),
                  torch.zeros(agent.num_layers, 1, agent.hidden_size, dtype=torch.float16).to(agent.device))

  episode_reward = 0
  episode_memory = deque(maxlen=agent.seq_len)
  done = False

  while not done:
    if render:
      agent.envs.render()

    state_tensor = torch.tensor(state2, dtype=torch.float32).to(agent.device).unsqueeze(0)

    if np.random.rand() <= agent.epsilon:
      action = agent.envs.action_space.sample()
    else:
      with torch.no_grad(), autocast(device_type=agent.device.type, dtype=torch.float16):
        q_values, hidden_state = agent.model(state_tensor, hidden_state)
        action = torch.argmax(q_values, dim=1).item()

    next_state, reward, done, _ = agent.envs.step(action)
    next_state0, next_state1, next_state2 = expand(next_state)
    episode_reward += reward

    episode_memory.append((state2, action, reward, next_state2, done, hidden_state))
    state0, state1, state2 = next_state0, next_state1, next_state2

    if len(episode_memory) >= agent.seq_len or done:
      train_on_batch(agent, episode_memory)
      episode_memory = deque(maxlen=agent.seq_len)

  agent.epsilon = max(agent.epsilon_min, agent.epsilon * agent.epsilon_decay)
  return episode_reward

# Define Train()

In [ ]:
def train(agent: TrainingAgent, num_episodes=1000, target_update_frequency=10, checkpoint_frequency=100, render=False, checkpoint_path="model_checkpoints"):
  total_rewards = []
  for episode in range(1, num_episodes + 1):
    episode_reward = run_episode(agent, render)
    total_rewards.append(episode_reward)

    if episode % target_update_frequency == 0:
      agent.target_model.load_state_dict(agent.model.state_dict())

    if episode % checkpoint_frequency == 0:
      import os
      os.makedirs(checkpoint_path, exist_ok=True)
      torch.save({
        'episode': episode,
        'model_state_dict': agent.model.state_dict(),
        'optimizer_state_dict': agent.optimizer.state_dict(),
        'reward': episode_reward,
        'epsilon': agent.epsilon
      }, f"{checkpoint_path}/checkpoint_{episode}.pt")

    if episode % 10 == 0:
      avg_reward = sum(total_rewards[-10:]) / 10
      print(f"Episode {episode}/{num_episodes}, Average Reward (Last 10): {avg_reward:.2f}, Epsilon: {agent.epsilon:.2f}")

  return total_rewards


# Hyper Parameters

### Neural Network Size

In [ ]:
input_size = 15
hidden_size = 50
output_size = 4

### Training Config

In [ ]:
seq_len = 30
batch_size = 1024
memory_size = 100_000

### Epsilon Decay Config

In [ ]:
epsilon = 1.0
epsilon_min = 0.01
epsilon_decay = 0.995

### Misc

In [ ]:
learning_rate = 1e-3
num_episodes = 1000
gamma = 0.99

# Initialization

In [ ]:
envs = gym.vector.AsyncVectorEnv([
    lambda: make_function(1),
    lambda: make_function(2),
    lambda: make_function(3)
])

agent = TrainingAgent(
  envs=envs,
  hidden_size=hidden_size,
  learning_rate=learning_rate,
  gamma=gamma,
  epsilon=epsilon,
  epsilon_min=epsilon_min,
  epsilon_decay=epsilon_decay,
  seq_len=seq_len,
  batch_size=batch_size,
  memory_size=memory_size
)

# Train

In [ ]:
total_rewards = train(agent, 100, 10, 1, True)